<img style="float: left; margin: 30px 15px 15px 15px;" src="https://pngimage.net/wp-content/uploads/2018/06/logo-iteso-png-5.png" width="300" height="100" /> 

<center><font color=#555555><font size=6> MAF1731 - Microstructure and Trading Systems </font> <br> <br>
    
<center><font color=#555555><font size=5> Maria Fernanda Amador Alvarez </font> <br> <br> <br> <br>
   
<center> <font color= #555555> <font size = 4> Jan 2023 </a> | <font color= #555555> <font size = 4> Repository: <a href= https://github.com/feramdor/Lab1> Link Repo

<center> <font color= #555555> <font size = 6> Módulo 1 </font> <br> <br> <font color= #555555> <font size = 5> Inversión del capital

# <font color= #6B6B6B> <font size = 5> Abstract </font>

Se tiene 1 millón de pesos y dos opciones para invertirlos. Una opción es hacer una inversión pasiva, comprando el 100% de un ETF que sigue el mercado de renta variable mexicano. La otra opción es hacer una inversión activa, creando un portafolio con los mismos activos que el ETF y haciendo un rebalanceo con un algoritmo específico.

# <font color= #6B6B6B> <font size = 6> 1. Introduction </font>

En este trabajo, se llevaron a cabo dos estrategias de inversión, una pasiva y una activa, para posteriormente comparar sus resultados y concluir cuál podría ser la mejor opción para un inversor que tiene un capital de 1 millón de pesos. Las acciones se tomaron de un índice creado con acciones pertenecientes a la Bolsa Mexicana de Valores, por lo que están listadas en pesos.

En la estrategia pasiva, se simula que se compran las mismas acciones que el índice NAFTRAC en la fecha 0 y no se hace ningún cambio en los títulos, se evalúa el valor de cada cartera mensual y se analizan sus rendimientos. Lo que se intenta simular en la estrategia activa es el rebalanceo del índice NAFTRAC, las ponderaciones de sus activos no son constantes y el índice cambia sus componentes de acuerdo con el desempeño de las compañías con el fin de diversificar sus inversiones y generar un mejor rendimiento.

Finalmente, se toman indicadores de rendimiento de cada estrategia, como el rendimiento promedio, el rendimiento acumulado y la relación Sharpe, para analizar cuál estrategia es mejor.

# <font color= #6B6B6B> <font size = 6 > 2. Install/Load Packages and Depedencies </font>

In [1]:
# Librerias
import pandas as pd
import numpy as np
import scipy.spatial.distance as sc
import matplotlib.pyplot as plt
import numpy as np